In [ ]:
!pip install newsapi-python

In [ ]:
!pip install ktrain

     |████████████████████████████████| 25.3MB 1.4MB/s 
     |████████████████████████████████| 6.8MB 43.9MB/s 
     |████████████████████████████████| 983kB 42.2MB/s 
     |████████████████████████████████| 266kB 37.7MB/s 
     |████████████████████████████████| 1.9MB 45.2MB/s 
     |████████████████████████████████| 1.2MB 51.0MB/s 
     |████████████████████████████████| 471kB 56.2MB/s 
     |████████████████████████████████| 870kB 38.6MB/s 
     |████████████████████████████████| 3.2MB 43.2MB/s 
  Created wheel for ktrain: filename=ktrain-0.26.2-cp37-none-any.whl size=25277794 sha256=999af1ea9f8d5fa373eb3bc603ebaf2cf16ce366910026f612ad11af54177010
  Stored in directory: /root/.cache/pip/wheels/67/2e/f1/c72afa08df8b2d984b910dea228902ce81dae4511afe9fafd2
  Created wheel for langdetect: filename=langdetect-1.0.8-cp37-none-any.whl size=993193 sha256=5b83d42a0371a536cf0974e235ee803b16a85fa4073d9b2a0e2e452e546de16f
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d4

In [ ]:
import os 
os.chdir("/content/drive/MyDrive")

In [ ]:
from newsapi import NewsApiClient

# Init
newsapi = NewsApiClient(api_key='e6cdd1bd9e3f473eae965351ded6c67f')

# /v2/top-headlines
top_headlines = newsapi.get_top_headlines(category='general',
                                          language='en',)

In [ ]:
top_headlines

{'articles': [{'author': None,
   'content': '"MarsHelicopter touchdown confirmed!" NASA\'s Jet Propulsion Laboratory tweeted Saturday.\r\nNASA\'s Ingenuity mini-helicopter has been dropped on the surface of Mars in preparation for its first flight,… [+2855 chars]',
   'description': "NASA's Ingenuity mini-helicopter has been dropped on the surface of Mars in preparation for its first flight, the US space agency said.",
   'publishedAt': '2021-04-04T08:06:43Z',
   'source': {'id': None, 'name': 'NDTV News'},
   'title': "NASA's Ingenuity Helicopter Dropped On Mars' Surface Ahead Of Flight - NDTV",
   'url': 'https://www.ndtv.com/world-news/nasas-ingenuity-helicopter-dropped-on-mars-surface-ahead-of-flight-2405848',
   'urlToImage': 'https://c.ndtvimg.com/2021-04/7rdjhl5_nasa-mars-helicopter-touchdown_625x300_04_April_21.jpg'},
  {'author': None,
   'content': '"We will move the court over it. This is unacceptable," Mamata Banerjee had said.\r\nNew Delhi: West Bengal Chief Minister Mama

In [ ]:
import pandas as pd
df = pd.DataFrame(top_headlines["articles"])

In [ ]:
df = df.drop(columns=["source", "author", "url", "urlToImage", "publishedAt"])

In [ ]:
df.head()

,title,description,content
0,Major railways to limit operations as personne...,Major Philippine railways will resume operatio...,Major Philippine railways will resume operatio...
1,Woman reported missing after boarding Interisl...,An hour after police issued a statement saying...,A woman reported missing after boarding an Int...
2,PH posts highest number of new COVID-19 recove...,"The Department of Health reported 41,205 new r...","MANILA - The Philippines confirmed 11,028 new ..."
3,Below the surface: reports of rising shark att...,Fatal shark encounters in Australia are higher...,SharksFatal shark encounters in Australia are ...
4,Fire crews battle scrub fire near Auckland CBD...,Trains have been stopped as emergency crews ba...,Trains have been stopped as emergency crews ba...


In [ ]:
df["data"] = df["title"] + " " + df["description"] + " " + df["content"]


In [ ]:
import ktrain
from ktrain import text

In [ ]:
predictor_load = ktrain.load_predictor('News Bias')

In [ ]:
arr = [df["data"].isna()]

In [ ]:
arr[0].values

array([False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False])

In [ ]:
biases = predictor_load.predict(df[~df["data"].isna()]["data"].values)

In [ ]:
bias_dict = {
    0 : "neutral",
    1: "right",
    2: "left"
}

In [ ]:
news_list = []
count = 0
for news, na in zip(top_headlines["articles"], arr[0].values):
  if ~na:
    news["bias"] = bias_dict[biases[count]]
    count += 1
    news_list.append(news)
news_list

[{'author': None,
  'bias': 'neutral',
  'content': '"MarsHelicopter touchdown confirmed!" NASA\'s Jet Propulsion Laboratory tweeted Saturday.\r\nNASA\'s Ingenuity mini-helicopter has been dropped on the surface of Mars in preparation for its first flight,… [+2855 chars]',
  'description': "NASA's Ingenuity mini-helicopter has been dropped on the surface of Mars in preparation for its first flight, the US space agency said.",
  'publishedAt': '2021-04-04T08:06:43Z',
  'source': {'id': None, 'name': 'NDTV News'},
  'title': "NASA's Ingenuity Helicopter Dropped On Mars' Surface Ahead Of Flight - NDTV",
  'url': 'https://www.ndtv.com/world-news/nasas-ingenuity-helicopter-dropped-on-mars-surface-ahead-of-flight-2405848',
  'urlToImage': 'https://c.ndtvimg.com/2021-04/7rdjhl5_nasa-mars-helicopter-touchdown_625x300_04_April_21.jpg'},
 {'author': None,
  'bias': 'neutral',
  'content': '"We will move the court over it. This is unacceptable," Mamata Banerjee had said.\r\nNew Delhi: West Bengal

In [ ]:
from newsapi import NewsApiClient
import ktrain
from ktrain import text
import pandas as pd
def get_biased_news(category):
  predictor_load = ktrain.load_predictor('News Bias')
  bias_dict = {
      0 : "neutral",
      1: "right",
      2: "left"
  }
  # Init
  newsapi = NewsApiClient(api_key='e6cdd1bd9e3f473eae965351ded6c67f')

  # /v2/top-headlines
  top_headlines = newsapi.get_top_headlines(category=category,
                                            country = "in",
                                            language='en',)
  
  df = pd.DataFrame(top_headlines["articles"])
  df = df.drop(columns=["source", "author", "url", "urlToImage", "publishedAt"])
  df["data"] = df["title"] + " " + df["description"] + " " + df["content"]
  arr = [df["data"].isna()]
  biases = predictor_load.predict(df[~df["data"].isna()]["data"].values)
  news_list = []
  count = 0
  for news, na in zip(top_headlines["articles"], arr[0].values):
    if ~na:
      news["bias"] = bias_dict[biases[count]]
      count += 1
      news_list.append(news)
  return {"articles" : news_list}


In [ ]:
  get_biased_news("general")

{'articles': [{'author': None,
   'bias': 'neutral',
   'content': '"MarsHelicopter touchdown confirmed!" NASA\'s Jet Propulsion Laboratory tweeted Saturday.\r\nNASA\'s Ingenuity mini-helicopter has been dropped on the surface of Mars in preparation for its first flight,… [+2855 chars]',
   'description': "NASA's Ingenuity mini-helicopter has been dropped on the surface of Mars in preparation for its first flight, the US space agency said.",
   'publishedAt': '2021-04-04T08:06:43Z',
   'source': {'id': None, 'name': 'NDTV News'},
   'title': "NASA's Ingenuity Helicopter Dropped On Mars' Surface Ahead Of Flight - NDTV",
   'url': 'https://www.ndtv.com/world-news/nasas-ingenuity-helicopter-dropped-on-mars-surface-ahead-of-flight-2405848',
   'urlToImage': 'https://c.ndtvimg.com/2021-04/7rdjhl5_nasa-mars-helicopter-touchdown_625x300_04_April_21.jpg'},
  {'author': None,
   'bias': 'neutral',
   'content': '"We will move the court over it. This is unacceptable," Mamata Banerjee had said.\r

In [ ]:
predictor_load = ktrain.load_predictor('News Bias')
category = "general"
bias_dict = {
    0 : "neutral",
    1: "right",
    2: "left"
}
# Init
newsapi = NewsApiClient(api_key='e6cdd1bd9e3f473eae965351ded6c67f')

# /v2/top-headlines
top_headlines = newsapi.get_top_headlines(category=category,
                                          country = "in",
                                          language='en',)

df = pd.DataFrame(top_headlines["articles"])
df = df.drop(columns=["source", "author", "url", "urlToImage", "publishedAt"])
df["data"] = df["title"] + " " + df["description"] + " " + df["content"]
biases = predictor_load.predict(df["data"].values)

news_list = []
for news, bias in zip(top_headlines["articles"], biases):
  news["bias"] = bias_dict[bias]
  news_list.append(news)

TypeError: ignored